In [8]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
model = ChatGroq(model="Gemma2-9b-It")

In [5]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    

In [7]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']
    
    # call llm generate outline
    prompt = f"Generate a detailed outline for a blog on the topic - {title}"
    outline = model.invoke(prompt).content
    
    # update state
    state['outline'] = outline

    return state


def create_blog(state: BlogState) -> BlogState:

    # fetch title and outline
    title = state['title']
    outline = state['outline']
    
    # call llm generate blog
    prompt = f"Write a detailed blog on the title - {title} \n\n using the following outline:\n {outline}"
    content = model.invoke(prompt).content
    
    # update state
    state['content'] = content

    return state

In [10]:
graph = StateGraph(BlogState)

# nodes
graph.add_node("create_outline",create_outline)
graph.add_node("create_blog",create_blog)

# edges
graph.add_edge(START,'create_outline')
graph.add_edge('create_outline','create_blog')
graph.add_edge('create_blog',END)


#compile

workflow = graph.compile()

In [11]:
initial_state = {'title': "Rise of AI in healthcare"}

final_state = workflow.invoke(initial_state)

print(final_state)

{'title': 'Rise of AI in healthcare', 'outline': "##  Rise of AI in Healthcare: A Detailed Blog Outline\n\n**I. Introduction:**\n\n*   Hook: Start with a captivating statistic or anecdote about AI in healthcare's potential impact.\n*   Define AI and its relevance to healthcare.\n*   Briefly outline the benefits and challenges of AI in healthcare.\n*   Thesis statement: AI is rapidly transforming healthcare, offering promising solutions while raising ethical considerations that need careful attention.\n\n**II. Current Applications of AI in Healthcare:**\n\n*   **Diagnostics:**\n    *   Image Analysis: AI's role in detecting diseases like cancer, Alzheimer's, and diabetic retinopathy through medical imaging.\n    *   Symptom Checkers: AI-powered chatbots and virtual assistants assisting with initial symptom assessments and triage.\n    *   Predictive Analytics: AI algorithms identifying patients at risk for specific conditions based on medical history and data.\n*   **Treatment:**\n    *

In [18]:
print(final_state['outline'])

##  Rise of AI in Healthcare: A Detailed Blog Outline

**I. Introduction:**

*   Hook: Start with a captivating statistic or anecdote about AI in healthcare's potential impact.
*   Define AI and its relevance to healthcare.
*   Briefly outline the benefits and challenges of AI in healthcare.
*   Thesis statement: AI is rapidly transforming healthcare, offering promising solutions while raising ethical considerations that need careful attention.

**II. Current Applications of AI in Healthcare:**

*   **Diagnostics:**
    *   Image Analysis: AI's role in detecting diseases like cancer, Alzheimer's, and diabetic retinopathy through medical imaging.
    *   Symptom Checkers: AI-powered chatbots and virtual assistants assisting with initial symptom assessments and triage.
    *   Predictive Analytics: AI algorithms identifying patients at risk for specific conditions based on medical history and data.
*   **Treatment:**
    *   Personalized Medicine: AI aiding in tailoring treatment plans b

In [16]:
print(final_state['content'])

## The Rise of AI in Healthcare: A Revolution in Care

Imagine a future where diseases are detected before symptoms even appear, where treatments are personalized to your unique genetic makeup, and where administrative tasks are handled effortlessly by intelligent machines. This isn't science fiction; it's the rapidly unfolding reality of artificial intelligence (AI) in healthcare. 

AI, the ability of computers to learn from data and perform tasks typically requiring human intelligence, is poised to revolutionize every aspect of healthcare delivery. From diagnostics to treatment, and even administrative processes, AI is offering promising solutions while simultaneously raising crucial ethical considerations.

**AI in Action: Transforming Healthcare Today**

AI is already making its mark across a spectrum of healthcare applications:

* **Diagnostics:** AI algorithms are excelling at analyzing medical images, detecting subtle patterns indicative of diseases like cancer, Alzheimer's, and